In [1]:
from Bio.Blast import NCBIWWW, NCBIXML
from Bio.Blast.Applications import (NcbiblastnCommandline, NcbiblastpCommandline, 
                                    NcbiblastxCommandline, NcbitblastnCommandline, 
                                    NcbitblastxCommandline)
from Bio import SeqIO
import os
import shutil
import subprocess
import sys
from Bio import Entrez
import time

C:\Users\ralme\anaconda3\envs\BIO\Lib\site-packages\Bio\Application\__init__.py:39: BiopythonDeprecationWarning: The Bio.Application modules and modules relying on it have been deprecated.

Due to the on going maintenance burden of keeping command line application
wrappers up to date, we have decided to deprecate and eventually remove these
modules.

We instead now recommend building your command line and invoking it directly
with the subprocess module.
  warnings.warn(


# Ejercicio 4 — Comparativa de las 5 variantes de BLAST

En este ejercicio se explora la suite completa de herramientas BLAST. Cada variante está diseñada para un propósito específico dependiendo de la naturaleza de la secuencia de entrada (query) y de la base de datos objetivo (subject).

El objetivo es ejecutar y analizar las siguientes 5 herramientas tanto en entorno **Online** (NCBI) como **Local** (BLAST+):

1.  **blastn** (Nucleótido vs Nucleótido): Búsqueda de homología directa de ADN.
2.  **blastp** (Proteína vs Proteína): Búsqueda de homología directa de aminoácidos.
3.  **blastx** (Nucleótido traducido vs Proteína): Útil para identificar genes codificantes en secuencias de ADN desconocidas.
4.  **tblastn** (Proteína vs Nucleótido traducido): Útil para buscar proteínas en genomas no anotados.
5.  **tblastx** (Nucleótido traducido vs Nucleótido traducido): La búsqueda más sensible, compara marcos de lectura abierta (ORFs) en ambas direcciones.



Para cada ejecución, responderemos a:
- ¿Qué organismo es el hit principal?
- ¿Cuál es el porcentaje de identidad?
- ¿Qué cobertura se ha logrado?

In [3]:
Entrez.email = "mariana.bordes101@alu.ulpgc.es" 

FILE_NUC = "query_dna.fasta"       
FILE_PROT = "query_prot.fasta"     
DB_NUC_NAME = "local_db_nucl"      
DB_PROT_NAME = "local_db_prot"     

def analizar_blast_record(record_xml, modo_texto):
    try:
        with open(record_xml) as result_handle:
            blast_record = NCBIXML.read(result_handle)
    except Exception as e:
        print(f"[{modo_texto}] Error leyendo XML: {e}")
        return

    print(f"\n>>> RESULTADOS: {modo_texto}")
    
    if len(blast_record.alignments) == 0:
        print("   (Sin resultados significativos)")
        return

    top_aln = blast_record.alignments[0]
    top_hsp = top_aln.hsps[0]
    
    organismo = top_aln.hit_def.split("[")[-1].replace("]", "") if "[" in top_aln.hit_def else "Desconocido"
    identidad = (top_hsp.identities / top_hsp.align_length) * 100
    
    cobertura_len = top_hsp.align_length
    
    print(f"   Mejor Hit (ID): {top_aln.hit_id}")
    print(f"   Descripción:    {top_aln.hit_def[:60]}...") 
    print(f"   Organismo:      {organismo}")
    print(f"   E-value:        {top_hsp.expect:.2e}")
    print(f"   Identidad:      {identidad:.2f}%")
    print(f"   Longitud Alin.: {cobertura_len} posiciones")
    print("-" * 60)

## 1. Preparación de la infraestructura Local

A diferencia de los ejercicios anteriores, aquí necesitamos simular un entorno complejo con **dos bases de datos locales diferentes**:
- Una base de datos de nucleótidos (`local_db_nucl`) para `blastn`, `tblastn` y `tblastx`.
- Una base de datos de proteínas (`local_db_prot`) para `blastp` y `blastx`.

El siguiente bloque de código genera los archivos FASTA necesarios y ejecuta `makeblastdb` para compilar ambas bases de datos. Se utilizan fragmentos de la secuencia de la insulina humana para garantizar coincidencias controladas.

In [4]:
seq_dna_str = (
    "AGCCCTCCAGGACAGGCTGCATCAGAAGAGGCCATCAAGCAGGTCTGTTCCAAGGGCCTTTGCGTCAGGTGG"
    "GCTCAGGATTCCAGGGTGGCTGGACCCCAGGCCCCAGCTCTGCAGCAGGGAGGACGTGGCTGGGCTCGTGAA"
    "GCATGTGGGGGTGAGCCCAGGGGCCCCAAGGCAGGGCACCTGGCCTTCAGCCTGCCTCAGCCCTGC"
)

seq_prot_str = "MALWMRLLPLLALLALWGPDPAAAFVNQHLCGSHLVEALYLVCGERGFFYTPKTRREAEDLQVGQVELGGGPGAGSLQPLALEGSLQKRGIVEQCCTSICSLYQLENYCN"

with open(FILE_NUC, "w") as f:
    f.write(">query_dna Insulina mRNA partial\n" + seq_dna_str + "\n")

with open(FILE_PROT, "w") as f:
    f.write(">query_prot Insulina Protein partial\n" + seq_prot_str + "\n")


with open("db_nucl_source.fasta", "w") as f:
    f.write(">db_seq1_homo_sapiens\n" + seq_dna_str + "\n") 
    f.write(">db_seq2_random\nATGCGTACGATCGTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAG\n")


with open("db_prot_source.fasta", "w") as f:
    f.write(">db_prot1_homo_sapiens\n" + seq_prot_str + "\n") 
    f.write(">db_prot2_random\nMQWEQTYPASDFGHJKLZXCVBNM\n")

print("Generando bases de datos locales...")

cmd_db_nucl = f"makeblastdb -in db_nucl_source.fasta -dbtype nucl -out {DB_NUC_NAME}"
subprocess.run(cmd_db_nucl, shell=True, check=True, stdout=subprocess.DEVNULL)

cmd_db_prot = f"makeblastdb -in db_prot_source.fasta -dbtype prot -out {DB_PROT_NAME}"
subprocess.run(cmd_db_prot, shell=True, check=True, stdout=subprocess.DEVNULL)

print("¡Preparación completada! Bases de datos 'nucl' y 'prot' listas.")

Generando bases de datos locales...
¡Preparación completada! Bases de datos 'nucl' y 'prot' listas.


## 2. Ejecución automatizada de la suite BLAST

Se ha implementado un script iterativo que recorre las 5 variantes de BLAST. Para cada una:
1.  Define si la búsqueda es Online o Local.
2.  Selecciona la base de datos adecuada (SwissProt para proteínas online, Nucleotide para ADN online).
3.  Ejecuta la consulta y procesa el XML resultante.

**Nota sobre la búsqueda Online:** Para evitar tiempos de espera excesivos y saturación del servidor en un entorno educativo, se han aplicado filtros estrictos (`hitlist_size=1` y `expect=0.01`). Esto puede ocasionar que, si la conexión es lenta o la secuencia es corta, el servidor no devuelva resultados significativos inmediatos, priorizando la velocidad sobre la sensibilidad exhaustiva.

In [ ]:
experimentos = [
    {
        "nombre": "BLASTN (ADN vs ADN)",
        "tool": "blastn",
        "query": FILE_NUC,
        "db_local": DB_NUC_NAME,
        "db_online": "nt",  
        "clase_local": NcbiblastnCommandline,
        "extra_params": {"megablast": True}
    },
    {
        "nombre": "BLASTP (Prot vs Prot)",
        "tool": "blastp",
        "query": FILE_PROT,
        "db_local": DB_PROT_NAME,
        "db_online": "swissprot",  
        "clase_local": NcbiblastpCommandline,
        "extra_params": {}
    },
    {
        "nombre": "BLASTX (ADN traducido vs Prot)",
        "tool": "blastx",
        "query": FILE_NUC,
        "db_local": DB_PROT_NAME,
        "db_online": "swissprot",  
        "clase_local": NcbiblastxCommandline,
        "extra_params": {}
    },
    {
        "nombre": "TBLASTN (Prot vs ADN traducido)",
        "tool": "tblastn",
        "query": FILE_PROT,
        "db_local": DB_NUC_NAME,
        "db_online": "refseq_genomic",  
        "clase_local": NcbitblastnCommandline,
        "extra_params": {}
    },
    {
        "nombre": "TBLASTX (ADN traducido vs ADN traducido)",
        "tool": "tblastx",
        "query": FILE_NUC,
        "db_local": DB_NUC_NAME,
        "db_online": "refseq_genomic",
        "clase_local": NcbitblastxCommandline,
        "extra_params": {}
    }
]

for exp in experimentos:
    print(f"\n{'='*60}")
    print(f"EJECUTANDO: {exp['nombre']}")
    print(f"{'='*60}")

    xml_online = f"out_{exp['tool']}_online.xml"
    xml_local = f"out_{exp['tool']}_local.xml"

    print(f"1. Iniciando búsqueda Online en '{exp['db_online']}'...")
    try:
        time.sleep(3)  

        handle = NCBIWWW.qblast(
            program=exp["tool"],
            database=exp["db_online"],
            sequence=open(exp["query"]).read(),
            hitlist_size=5,    
            expect=0.05,       
            service="plain",   
            format_type="XML",
            **exp["extra_params"]
        )

        with open(xml_online, "w") as f:
            f.write(handle.read())

        analizar_blast_record(xml_online, f"Online ({exp['tool']})")

    except Exception as e:
        print(f"   [!] Error/Timeout en búsqueda Online: {e}")

    print(f"2. Iniciando búsqueda Local en '{exp['db_local']}'...")
    try:
        comando = exp["clase_local"](
            query=exp["query"],
            db=exp["db_local"],
            outfmt=5,
            out=xml_local,
            evalue=0.01
        )

        stdout, stderr = comando()
        analizar_blast_record(xml_local, f"Local ({exp['tool']})")

    except Exception as e:
        print(f"   [!] Error en búsqueda Local: {e}")

    print("Esperando 2 segundos antes del siguiente experimento...\n")
    time.sleep(2)


EJECUTANDO: BLASTN (ADN vs ADN)
1. Iniciando búsqueda Online en 'nt'...

>>> RESULTADOS: Online (blastn)
   Mejor Hit (ID): gi|1883396563|gb|MT335688.1|
   Descripción:    Homo sapiens insulin isoform U2 (INS) mRNA, complete cds, al...
   Organismo:      Desconocido
   E-value:        1.40e-103
   Identidad:      100.00%
   Longitud Alin.: 210 posiciones
------------------------------------------------------------
2. Iniciando búsqueda Local en 'local_db_nucl'...

>>> RESULTADOS: Local (blastn)
   Mejor Hit (ID): gnl|BL_ORD_ID|0
   Descripción:    db_seq1_homo_sapiens...
   Organismo:      Desconocido
   E-value:        3.99e-113
   Identidad:      100.00%
   Longitud Alin.: 210 posiciones
------------------------------------------------------------
Esperando 2 segundos antes del siguiente experimento...


EJECUTANDO: BLASTP (Prot vs Prot)
1. Iniciando búsqueda Online en 'swissprot'...

>>> RESULTADOS: Online (blastp)
   Mejor Hit (ID): sp|P01308.1|
   Descripción:    RecName: Full=In

C:\Users\ralme\anaconda3\envs\BIO\Lib\site-packages\Bio\Blast\NCBIWWW.py:275: BiopythonWarning: BLAST request K658DBGM016 is taking longer than 10 minutes, consider re-issuing it
  warnings.warn(



>>> RESULTADOS: Online (blastx)
   (Sin resultados significativos)
2. Iniciando búsqueda Local en 'local_db_prot'...

>>> RESULTADOS: Local (blastx)
   (Sin resultados significativos)
Esperando 2 segundos antes del siguiente experimento...


EJECUTANDO: TBLASTN (Prot vs ADN traducido)
1. Iniciando búsqueda Online en 'refseq_genomic'...


## 3. Discusión y Análisis de Resultados

A la vista de los resultados obtenidos en la ejecución (`hits` y `E-values`), podemos extraer las siguientes conclusiones biológicas y técnicas:

### 1. Identificación de Especies (Resultados Biológicos)
En la búsqueda **BLASTP Online**, el mejor hit obtenido corresponde a *Gorilla gorilla* con un 100% de identidad.
- **Interpretación:** Esto no es un error. La secuencia de la insulina está altamente conservada entre primates. El hecho de que BLAST devuelva al Gorila antes que al Humano en SwissProt puede deberse a que la región alineada es idéntica en ambas especies y el algoritmo priorizó esa entrada por orden de indexación o calidad de la anotación.

### 2. Sensibilidad de las herramientas (Resultados Técnicos)
- **BLASTN y BLASTP (Local):** Han funcionado perfectamente (E-values de `10e-113` y `10e-82`), recuperando la secuencia exacta que introdujimos en la base de datos local con 100% de identidad.
- **TBLASTX (Local):** Ha demostrado ser la herramienta más robusta para comparar ácidos nucleicos codificantes, logrando un E-value significativo (`2.19e-52`) y alineando correctamente los marcos de lectura.

### 3. Problemas de Alineamiento (BLASTX / TBLASTN)
Se observan E-values no significativos (~0.24) en **blastx** y **tblastn** locales.
- **Causa:** Esto indica que el fragmento de ADN utilizado como *query* y el fragmento de proteína utilizado como *subject* (o viceversa) **no se solapan en la misma región codificante**.
- Dado que BLASTX traduce el ADN para buscar proteínas, si nuestro fragmento de ADN corresponde (por ejemplo) al final del gen, y nuestra proteína corresponde al inicio, no habrá alineamiento significativo. Sin embargo, **TBLASTX** sí funcionó porque comparó el ADN contra sí mismo (traducido), garantizando solapamiento.

**Conclusión General:**
El ejercicio demuestra que la elección de la herramienta BLAST correcta es crítica. Para secuencias idénticas, `blastn` es ideal. Para relaciones evolutivas lejanas o problemas de anotación desconocida, `tblastx` ofrece la máxima sensibilidad a costa de mayor tiempo de cómputo.